In [1]:
# coding=utf8

# pip install accelerate
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
)


In [1]:
import json

test_file = "./test.zh-en.news.json"
data = []
with open(test_file,'r',encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))
inputs = []
for d in data:
    inputs.append(d['translation']['zh'])
# print(data[0])

In [2]:
outputs = []

for d in data:
    outputs.append(d['translation']['en'])
   
with open("./official.txt", "w") as f:
    for item in outputs:
        f.write(item.strip().replace("\n",'') + "\n")

In [5]:
model_name_or_path = "./Qwen2-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,padding_side='left')

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from tqdm import tqdm

batch_size = 64
responses = []
for i in tqdm(range(0,len(inputs),batch_size),desc="Make Batch",unit="batch"):
    middle_inputs = []
    for prompt in inputs[i:i+batch_size]:
        four_shot = """Zh:"是否有途径处罚他"\nEn:"Is there a way to punish him?"\n\nZh:"以免再次发生这样的事情"\nEn:"So that such a thing won’t happen again."\n\nZh:"用空气来洗手，哈口气判断疾病，精确搜寻雾霾来源 昨天晚上，在浙江大学2016年学术年会开幕式上，一系列脑洞大开的学术成果获得表彰。"\nEn:"Washing hands with air, diagnosing disease through breath, and accurately seeking for source of smog; last night, a series of creative academic achievements were awarded at the opening ceremony of the 2016 Annual Academic Conference of Zhejiang University."\n\nZh:"生物传感器：你的家庭体检医生"\nEn:"Biosensor: your home physical examination doctor"\n\n"""
        messages = [
            {"role": "system", "content": "you are a helpful assistant"},
            {"role": "user", "content": f"{four_shot}Zh:{prompt}\En:"},
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        middle_inputs.append(text)

    model_inputs = tokenizer(middle_inputs,return_tensors="pt",padding = True)
    
    generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
    )

    generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

    batch_responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    responses.extend(batch_responses)
    
with open("./responses.txt", "w") as f:
    for item in responses:
        f.write(item.strip().replace("\n",'') + "\n")

Make Batch: 100%|██████████| 1875/1875 [4:37:30<00:00,  8.88s/batch]     


The batchsize have significant influence on the proformance of the model.  
In the case of different batchsize, tokenizer need have different padding length.
if the pad token is too long ,the orignal sentence will not get the well understand by model.
***batch_size - 3***
```output:batch_size = 3
1:是否有途径处罚他
1:system
Translate the following content into English.Don't add any other information.
user
***是否有途径处罚他***
assistant
"Does he have any other penalty?"
len of inputs_id:43	len of responses_id:56
2:以免再次发生这样的事情
2:system
Translate the following content into English.Don't add any other information.
user
***以免再次发生这样的事情***
assistant
"Please do not repeat anything else."
len of inputs_id:43	len of responses_id:56
3:又或者餐厅还要多久会准备好？可不可以帮我问下？
3:system
Translate the following content into English.Don't add any other information.
user
***又或者餐厅还要多久会准备好？可不可以帮我问下？***
assistant
Could you please tell me when the restaurant will be ready?
len of inputs_id:43	len of responses_id:56
```

In [5]:
import gradio as gr 
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name_or_path = "../models/Qwen2-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,padding_side='left')

def get_one_response(Input):
    messages = [
            {"role": "system", "content": "you are a helpful assistant"},
            {"role": "user", "content": f"{Input}"},
        ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    middle_inputs = []
    middle_inputs.append(text)

    model_inputs = tokenizer(middle_inputs,return_tensors="pt",padding = True)
    
    generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
    )

    generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

    batch_responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return batch_responses[0]


def greet(Input):
    return get_one_response(Input)

demo = gr.Interface(fn=greet, 
                    inputs=["text"], 
                    outputs="text",
                    title="ChatBot",
                    description="A ChatBot Based on Qwen2-0.5B-Instruction",
                    article="ShAn3003",
)

demo.launch()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
